In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.signal import detrend
import os
import pickle

# ==============================
# Step 1: Load and Preprocess Data
# ==============================
def load_and_preprocess_data(file_path, target_column, sequence_length, scaler_type='StandardScaler', detrend_data=False):
    # Load data
    data = pd.read_csv(file_path)

    # Detrend the data if required (e.g., for Battery Level)
    if detrend_data:
        data[target_column] = detrend(data[target_column])

    # Normalize the target column
    if scaler_type == 'MinMaxScaler':
        scaler = MinMaxScaler()
    else:
        scaler = StandardScaler()

    data[target_column] = scaler.fit_transform(data[[target_column]])

    # Create sequences
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[target_column].iloc[i:i + sequence_length].values
        sequences.append(seq)

    sequences = np.array(sequences)
    sequences = np.expand_dims(sequences, axis=2)  # Add feature dimension
    return sequences, scaler

# ==============================
# Step 2: Define LSTM Autoencoder
# ==============================
def create_lstm_autoencoder(sequence_length, input_dim, latent_dim=32):
    # Encoder
    input_layer = Input(shape=(sequence_length, input_dim))
    encoded = LSTM(latent_dim, activation='relu', return_sequences=False)(input_layer)
    encoded = Dense(latent_dim // 2, activation='relu')(encoded)

    # Decoder
    decoded = RepeatVector(sequence_length)(encoded)
    decoded = LSTM(latent_dim, activation='relu', return_sequences=True)(decoded)
    decoded = TimeDistributed(Dense(input_dim))(decoded)

    # Autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer='adam', loss='mae')
    return autoencoder

# ==============================
# Step 3: Train and Save Models
# ==============================
def train_and_save_model(sequences, model_path, sequence_length, input_dim, latent_dim=32, epochs=50, batch_size=32):
    # Define LSTM Autoencoder
    model = create_lstm_autoencoder(sequence_length, input_dim, latent_dim)

    # Train model with early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(sequences, sequences, epochs=epochs, batch_size=batch_size, validation_split=0.2, shuffle=True, callbacks=[early_stopping])

    # Save model
    model.save(model_path)
    print(f"Model saved to {model_path}")
    return model

# ==============================
# Step 4: Main Script
# ==============================
if __name__ == "__main__":
    # Parameters
    file_path = "C:/Users/saura/OneDrive/Desktop/data_streaming/data_generators/srinagar/dwlrdata_srinagar_final(1).csv"
    sequence_length = 30         # Number of timesteps per sequence
    epochs = 50                  # Training epochs
    batch_size = 32              # Batch size
    output_dir = "C:/Users/saura/Downloads/trained_model_v2"  # Directory to save models

    # Create output directory if not exists
    os.makedirs(output_dir, exist_ok=True)

    # Depth
    print("Training model for Depth...")
    depth_sequences, depth_scaler = load_and_preprocess_data(
        file_path, target_column="Depth", sequence_length=sequence_length, scaler_type='StandardScaler'
    )
    train_and_save_model(depth_sequences, os.path.join(output_dir, "depth_model_srinagar.h5"), sequence_length, input_dim=1, latent_dim=32, epochs=epochs, batch_size=batch_size)

    # Temperature
    print("Training model for Temperature...")
    temperature_sequences, temperature_scaler = load_and_preprocess_data(
        file_path, target_column="Temperature", sequence_length=sequence_length, scaler_type='StandardScaler'
    )
    train_and_save_model(temperature_sequences, os.path.join(output_dir, "temperature_model_srinagar.h5"), sequence_length, input_dim=1, latent_dim=32, epochs=epochs, batch_size=batch_size)

    # Battery Level
    print("Training model for Battery Level...")
    battery_sequences, battery_scaler = load_and_preprocess_data(
        file_path, target_column="Battery Level", sequence_length=sequence_length, scaler_type='MinMaxScaler', detrend_data=True
    )
    train_and_save_model(battery_sequences, os.path.join(output_dir, "battery_model_srinagar.h5"), sequence_length, input_dim=1, latent_dim=64, epochs=epochs, batch_size=batch_size)

    # Save scalers for future use
    scalers = {
        "Depth": depth_scaler,
        "Temperature": temperature_scaler,
        "Battery Level": battery_scaler
    }
    scaler_path = os.path.join(output_dir, "scalers_srinagar.pkl")
    with open(scaler_path, "wb") as f:
        pickle.dump(scalers, f)
    print(f"Scalers saved to {scaler_path}")

    print("Training complete. Models and scalers are saved.")


Training model for Depth...
Epoch 1/50
730/730 [==============================] - 18s 17ms/step - loss: 0.1009 - val_loss: 0.0527
Epoch 2/50
730/730 [==============================] - 13s 17ms/step - loss: 0.0468 - val_loss: 0.0519
Epoch 3/50
730/730 [==============================] - 14s 19ms/step - loss: 0.0412 - val_loss: 0.0390
Epoch 4/50
730/730 [==============================] - 14s 19ms/step - loss: 0.0327 - val_loss: 0.0319
Epoch 5/50
730/730 [==============================] - 14s 19ms/step - loss: 0.0304 - val_loss: 0.0313
Epoch 6/50
730/730 [==============================] - 14s 19ms/step - loss: 0.0289 - val_loss: 0.0285
Epoch 7/50
730/730 [==============================] - 14s 19ms/step - loss: 0.0277 - val_loss: 0.0252
Epoch 8/50
730/730 [==============================] - 14s 19ms/step - loss: 0.0266 - val_loss: 0.0309
Epoch 9/50
730/730 [==============================] - 13s 18ms/step - loss: 0.0250 - val_loss: 0.0218
Epoch 10/50
730/730 [==============================] -

D:\anaconda\envs\anomaly_detection\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
730/730 [==============================] - 16s 17ms/step - loss: 0.0839 - val_loss: 0.0374
Epoch 2/50
730/730 [==============================] - 12s 16ms/step - loss: 0.0245 - val_loss: 0.0307
Epoch 3/50
730/730 [==============================] - 12s 16ms/step - loss: 0.0224 - val_loss: 0.0310
Epoch 4/50
730/730 [==============================] - 12s 17ms/step - loss: 0.0207 - val_loss: 0.0323
Epoch 5/50
730/730 [==============================] - 12s 17ms/step - loss: 0.0209 - val_loss: 0.0248
Epoch 6/50
730/730 [==============================] - 12s 16ms/step - loss: 0.0207 - val_loss: 0.0296
Epoch 7/50
730/730 [==============================] - 12s 17ms/step - loss: 0.0222 - val_loss: 0.0263
Epoch 8/50
730/730 [==============================] - 12s 16ms/step - loss: 0.0204 - val_loss: 0.0248
Epoch 9/50
730/730 [==============================] - 12s 16ms/step - loss: 0.0200 - val_loss: 0.0296
Epoch 10/50
730/730 [==============================] - 11s 16ms/step - loss: 0.019

D:\anaconda\envs\anomaly_detection\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
730/730 [==============================] - 17s 19ms/step - loss: 0.0417 - val_loss: 0.0176
Epoch 2/50
730/730 [==============================] - 14s 19ms/step - loss: 0.0177 - val_loss: 0.0174
Epoch 3/50
730/730 [==============================] - 14s 19ms/step - loss: 0.0123 - val_loss: 0.0132
Epoch 4/50
730/730 [==============================] - 14s 19ms/step - loss: 0.0094 - val_loss: 0.0088
Epoch 5/50
730/730 [==============================] - 14s 19ms/step - loss: 0.0076 - val_loss: 0.0090
Epoch 6/50
730/730 [==============================] - 14s 19ms/step - loss: 0.0071 - val_loss: 0.0041
Epoch 7/50
730/730 [==============================] - 16s 22ms/step - loss: 0.0063 - val_loss: 0.0103
Epoch 8/50
730/730 [==============================] - 16s 21ms/step - loss: 0.0061 - val_loss: 0.0051
Epoch 9/50
730/730 [==============================] - 15s 21ms/step - loss: 0.0062 - val_loss: 0.0055
Epoch 10/50
730/730 [==============================] - 15s 21ms/step - loss: 0.005

D:\anaconda\envs\anomaly_detection\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
